In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

2023-11-11 23:53:01.442609: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-11 23:53:01.685707: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-11 23:53:01.685743: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-11 23:53:01.686677: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-11 23:53:01.790222: I tensorflow/core/platform/cpu_feature_g

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Número de GPUs disponíveis:", len(physical_devices))

Número de GPUs disponíveis: 1


2023-11-11 23:53:05.043440: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-11 23:53:05.066459: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-11 23:53:05.066508: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [4]:
from tensorflow.keras.preprocessing import sequence

In [6]:
import pandas as pd
import numpy as np

In [7]:
seed = 777
np.random.seed(seed)

In [8]:
df_1 = pd.read_csv('../data/BERT_Embeddings_1.csv')
df_2 = pd.read_csv('../data/BERT_Embeddings_2.csv')

df = pd.concat([df_1,df_2])
df.reset_index(inplace=True)

In [118]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset


embeddings = df.iloc[:,8:].values
labels = df['Class']

# Supondo que você tenha uma coluna 'rotulo' em seu DataFrame
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=seed)

# Convertendo para tensores do PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Criando datasets e dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [147]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob=0.5):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# Defina os parâmetros do modelo
input_size = 768  # número de embeddings
hidden_size = 16  # tamanho do estado oculto
output_size = 3  # substitua num_classes pelo número real de classes

# Crie uma instância do modelo
model = LSTMModel(input_size, hidden_size, output_size)


In [148]:
import torch.optim as optim

# Defina a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento
num_epocas = 500

for epoca in range(num_epocas):
    model.train()
    optimizer.zero_grad()

    saida = model(X_train_tensor)
    perda = criterion(saida, y_train_tensor)
    
    perda.backward()
    optimizer.step()

    print(f'Época [{epoca + 1}/{num_epocas}], Perda: {perda.item():.4f}')



Época [1/500], Perda: 1.1086
Época [2/500], Perda: 1.1003
Época [3/500], Perda: 1.0985
Época [4/500], Perda: 1.0962
Época [5/500], Perda: 1.0964
Época [6/500], Perda: 1.0943
Época [7/500], Perda: 1.0949
Época [8/500], Perda: 1.0948


Época [9/500], Perda: 1.0916
Época [10/500], Perda: 1.0914
Época [11/500], Perda: 1.0891
Época [12/500], Perda: 1.0882
Época [13/500], Perda: 1.0876
Época [14/500], Perda: 1.0876
Época [15/500], Perda: 1.0846
Época [16/500], Perda: 1.0865
Época [17/500], Perda: 1.0869
Época [18/500], Perda: 1.0831
Época [19/500], Perda: 1.0819
Época [20/500], Perda: 1.0822
Época [21/500], Perda: 1.0799
Época [22/500], Perda: 1.0794
Época [23/500], Perda: 1.0783
Época [24/500], Perda: 1.0771
Época [25/500], Perda: 1.0719
Época [26/500], Perda: 1.0767
Época [27/500], Perda: 1.0773
Época [28/500], Perda: 1.0750
Época [29/500], Perda: 1.0736
Época [30/500], Perda: 1.0659
Época [31/500], Perda: 1.0710
Época [32/500], Perda: 1.0676
Época [33/500], Perda: 1.0676
Época [34/500], Perda: 1.0678
Época [35/500], Perda: 1.0679
Época [36/500], Perda: 1.0658
Época [37/500], Perda: 1.0667
Época [38/500], Perda: 1.0630
Época [39/500], Perda: 1.0629
Época [40/500], Perda: 1.0643
Época [41/500], Perda: 1.0595
Época [42/5

In [149]:
# Avaliação no conjunto de teste
model.eval()

with torch.no_grad():
    saida_treino = model(X_train_tensor)
    _, predicoes_treino = torch.max(saida_treino, 1)
    acuracia_treino = torch.sum(predicoes_treino == y_train_tensor).item() / len(y_train_tensor)

    saida_teste = model(X_test_tensor)
    _, predicoes_teste = torch.max(saida_teste, 1)
    acuracia_teste = torch.sum(predicoes_teste == y_test_tensor).item() / len(y_test_tensor)

In [150]:
print(acuracia_treino)
print(acuracia_teste)

0.5647916666666667
0.525
